# Lanczos, the basics

The Lanczos algorithm is an iterative method to find $p$ eigenvalues/eigenvectors of an Hermitian matrix $A$ of dimension $n$. It does so as follows

1. Find a tridiagonal matrix $T$ of dimension $p$ and an orthonormal matrix $V$ of dimensions $n x p$ such $A = V^H T V$
2. Find the eigenvalues / eigenvectors of $T$ through typical dense methods ($p << n$). Those are called ritz values / ritz vectors of $A$.
3. Find eigenvalues / eigenvectors of $A$ from the ritz values/vectors

In [1]:
import numpy as np
from scipy.linalg import eig, qr, hessenberg
from scipy.stats import ortho_group

from matplotlib import pyplot as plt

import seaborn as sns

sns.set_theme()
np.set_printoptions(precision=4)

In [46]:
np.random.seed(42)

n = 1000

diag_values = np.arange(1, n+1, dtype=np.double)[::-1]
D = np.diag(diag_values)

# Build a diagonalizable, symmetric real matrix
Q = ortho_group.rvs(dim=n)
A = Q @ D @ Q.T

nev = 6

niter_default = min(n, max(2*nev + 1, 20))
niter = 500

q = np.empty((n, niter), dtype=A.dtype)

q[:, 0] = np.random.randn(n)
q[:, 0] /= np.linalg.norm(q[:, 0])

tol = np.finfo(A.dtype).eps * 100

# A = V^H T V is of the form
# alpha0...alpha n-1 is the diagonal
# beta1 ... betan is the lower/upper diagonal
w = np.zeros(n, dtype=A.dtype)
alpha = np.zeros(niter, dtype=A.dtype)
beta = np.zeros(niter, dtype=A.dtype)

q0 = q[:, 0]
w[:] = A @ q0
alpha[0] = w.T @ q0
w -= alpha[0] * q0

for k in range(1, niter):
    qk = q[:, k]
    qkp = q[:, k-1]
    
    beta[k] = np.linalg.norm(w)
    if beta[k] < tol:
        raise ValueError("Resample v0 not implemented")
    else:
        qk[:] = w / beta[k]    
    w[:] = A @ qk
    alpha[k] = w.T @ qk
    w -= alpha[k] * qk + beta[k] * qkp 

In [47]:
print(beta)
T = np.zeros((niter, niter), dtype=A.dtype)
T[np.arange(niter), np.arange(niter)] = alpha[:niter]
if niter > 1:
    T[np.arange(niter-1) + 1, np.arange(niter-1)] = beta[1:niter+1]
    T[np.arange(niter-1), np.arange(niter-1) + 1] = beta[1:niter+1]

#print(alpha)
#print(beta)
#print(T)
print(sorted(np.linalg.eig(T)[0])[::-1][:nev])
print(sorted(np.linalg.eig(A)[0])[::-1][:nev])

[  0.     286.7824 252.2688 252.7154 247.4592 253.3722 251.7077 253.7042
 247.6698 256.8008 253.2803 251.0088 238.3765 256.1805 250.7874 247.3579
 247.7476 250.7957 245.2004 251.0782 253.303  240.0569 251.8474 249.8736
 253.2693 255.353  248.1454 247.9539 248.7226 248.8441 248.0078 252.2763
 244.1284 253.2076 245.3391 248.5772 258.4096 247.813  247.3319 255.778
 253.9907 246.6489 238.1068 260.7095 246.9179 243.1456 246.6545 252.4054
 249.9803 254.9785 252.0529 246.5646 240.6226 252.2615 247.953  250.1273
 257.5768 248.0161 244.429  252.2731 242.8259 246.7749 253.9429 249.4974
 243.9265 252.1294 253.7396 247.0332 259.4744 249.6194 250.5141 239.1316
 249.694  253.1816 243.6165 246.5519 249.2795 246.3538 256.5886 241.2843
 259.9395 250.1738 240.5212 251.4158 241.0508 252.3933 254.4864 244.419
 243.1597 241.689  254.8052 243.1221 258.6412 249.7778 241.3639 254.079
 250.1254 245.3776 242.231  251.8833 252.0195 238.3887 246.1001 257.3456
 244.2305 251.3268 248.9095 241.0375 241.4649 252.0312

In [39]:
from scipy.sparse.linalg import eigs
eigs(A, nev)[0]

array([1000.+0.j,  999.+0.j,  998.+0.j,  997.+0.j,  996.+0.j,  995.+0.j])